# AC cal/ECEmeasurement

## define packages and readings

In [1]:
# import all necessary packages and drivers
import numpy as np
import pandas as pd
import re
from drivers.keithley6221 import Keithley6221
from drivers.keithley2182 import Keithley2182
from drivers.keithley2182a import Keithley2182A
from drivers.andeenHagerling import AndeenHagerling

from drivers.ppms_server import PPMS

# from drivers.MFLI import MFLI
from drivers.sr830 import Sr830
from drivers.sr860 import Sr860
from drivers.rp100 import Rp100
import time
import matplotlib.dates as md
from functools import  partialmethod,partial
from main_rp import (start_process, stop_process,pause_process,resume_process, 
                                  switch_file, clear_plot,update_PID)
# import matplotlib
from utility import (cls_all, error_response,get_d,get_disp,get_current_disp,
                     get_current_strain,wait_until,get_highest_indx_file, reset_all,get_cernox_temp)
# global data_to_save
from read_write_tab import read_file,read_idx,read_max_idx
%matplotlib inline

### Init instr

In [2]:
# initialize all instruments needed

pm = PPMS()
Sr830_1 = Sr830(gpib_addr=9,name='SR830-1') #longitutional
Sr830_2 = Sr830(gpib_addr=14,name='SR830-2') #Hall
rp100 = Rp100(serial_port=7,name='RP100')
ah = AndeenHagerling(gpib_addr=28,name='AH')
kl6221 = Keithley6221(gpib_addr=12,name='KL6221')

# mfli_5793 = MFLI(server_host='192.168.1.10')
# kl2182 = Keithley2182(gpib_addr=22,name='KL2182')
# kl2182a_1 = Keithley2182A(gpib_addr=2,name='KL2182A1')
# kl2182a_2 = Keithley2182A(gpib_addr=6,name='KL2182A2')
# kl2182a_3 = Keithley2182A(gpib_addr=7,name='KL2182A3')
# kl6220 = Keithley6221(gpib_addr=23,name='KL6220')
gpibInstrumentList = [
    # pm,
    Sr830_1,Sr830_2,
    rp100,ah,
    kl6221,
    
]
allInstrumentList = [pm,]+ gpibInstrumentList \
# + [mfli_5793]

### Clear all

In [3]:
clear_funcs = [
    
    pm.cls_ppms,
    Sr830_1.cls,
    Sr830_2.cls,
    # Sr860_1.cls,
#     kl2182.cls,
#     kl2182a_1.cls,
#     kl2182a_2.cls,
#     kl2182a_3.cls,
    # ah.cls,rp100.cls,
    kl6221.cls
#     kl6220.cls
]


In [4]:
cls_all(clear_funcs)

SR830-1 Initializaing resource manager
SR830-2 Initializaing resource manager
KL6221 Initializaing resource manager


### Set measurement

In [5]:
if_plot = False
if True: #if_plot use
    #plot configuration
    ncol, nrow = 3,3
    #all plot x,y s
    plot_axes_col_names = [
        ['sys:time','PPMS:T_pm'],
        ['sys:time','PPMS:H_pm'],
        ['sys:time','AH:C_d'],
        ['sys:time','SR830-1:X'],
        ['sys:time','SR830-1:Y'],
        ['RP100:setVolt1_now','RP100:setVolt2_now'],
        ['sys:time','SR830-2:X'],
        ['sys:time','SR830-2:Y'],
        ['PPMS:T_pm','SR830-1:X'],
        # ['PPMS:T_pm','dev5793:0:X'],
        # ['PPMS:T_pm','dev5793:0:Y'],
        
        # ['sys:time','AH:C_d'],
        # ['AH:C_d','dev5793:0:X'],
        # ['AH:C_d','dev5793:0:Y'],
        # ['AH:C_d','SR860-1:X'],
        # ['AH:C_d','SR860-1:Y'],
        
        
        
    #     ['SR830-1:X','SR830-1:Y'],
    #     ['KL2182:V','KL2182A1:V'],
        # ['RP100:setVolt1_now','RP100:setVolt2_now'],
        # ['SR830-1:Freq','SR860-1:Theta'],
        # ['SR830-1:Freq','SR860-1:Y'],
        # ['SR830-1:Freq','dev5793:0:Theta'],
        # ['SR830-1:Freq','dev5793:0:R'],
    ]

#data saving
data_folder = 'C:/Data/Zhenhai/strainedEU-052925-S2'
default_file = 'test_data_file' # first file to save

#all data entries saved in files. format as "instrument_name:reading_name" reading name can be reset in drivers
read_col_names = [
    # sys:time,
    'PPMS:T_pm',
    'PPMS:SetTemp',
    'PPMS:H_pm',
    'PPMS:P_pm',
    'PPMS:P_unit',
    'PPMS:SetField',
    'SR830-1:X',
    'SR830-1:Y',
    'SR830-1:R',
    'SR830-1:Theta',
    'SR830-1:Freq',
    'SR830-1:Amp',
    'SR830-2:X',
    'SR830-2:Y',
    'SR830-2:R',
    'SR830-2:Theta',
    'SR830-2:Freq',
    'SR830-2:Amp',
    'AH:C_d',
    'RP100:setVolt1_now',
    'RP100:setVolt2_now',
    'RP100:measVolt1',
    'RP100:measVolt2',
    'RP100:Output1',
    'RP100:Output2',
    # 'dev5793:0:X',
    # 'dev5793:0:Y',
    # 'dev5793:0:R',
    # 'dev5793:0:Theta',
    # 'dev5793:0:Freq',
    'KL6221:Iamp',
    'KL6221:Freq',
    'KL6221:Iac_on'
    # 'KL6220:Iout',
    # 'KL6220:Iout_on'
]

#reading functions are defined in drivers
read_funcs = [
    pm.get_temp,
    pm.get_field,
    pm.get_set_field,
    pm.get_set_temp,
    pm.get_pressure,
    Sr830_1.partial_auto_read,
    Sr830_1.get_amplitude,
    Sr830_1.get_frequency,
    Sr830_2.partial_auto_read,
    Sr830_2.get_amplitude,
    Sr830_2.get_frequency,
#     kl2182.partial_fetch,
#     kl2182a_1.partial_fetch,
#     kl2182a_2.partial_fetch,
#     kl2182a_3.partial_fetch,
    ah.trigger_and_read,
    rp100.source_value,
    rp100.get_set_source_value,
    rp100.get_meas_source_value,
    rp100.get_output,
#     kl6220.get_source_value,
#     kl6220.get_output
    # mfli_5793.get_data,
    kl6221.get_ac_source,
    kl6221.get_ac_output
]



PID_control_map = {
    'strain':[
    [['AH:C_d'],ah.trigger_and_read],
    [['RP100:setVolt1_now', 'RP100:setVolt2_now'], rp100.set_source_value]
    ],
}

init_PID_value = {
    'strain': None,
}

# test your reading functions, no data selection
def read_all(funcs_to_call,read_col_names=read_col_names):
        all_data = []
        all_data_name = []
        for i in range(len(funcs_to_call)):
            t1 = time.time()
            data,name = error_response(funcs_to_call[i])
            print(i,time.time()-t1)
            print(i,name,data,{key:value for key,value in zip(name,data)})
            all_data += data
            all_data_name += name
        return np.array(all_data), np.array(all_data_name)
t = time.time()
all_data, all_data_name = read_all(read_funcs)
print('overall takes',time.time()-t)
if any([not (x in all_data_name) for x in read_col_names]):
    print('columns not in reading')
else:
    print('all columns read')

# if any([not (x in read_col_names) for x in np.array(plot_axes_col_names).flatten()]):
#     print('columns not in reading')
# else:
#     print('all plot axis recorded')
print({key:value for key,value in zip(all_data_name,all_data,)})
reset_all(allInstrumentList)
# for it in gpibInstrumentList:
#     print(it._resource_manager)

0 0.0019676685333251953
0 ['PPMS:T_pm', 'PPMS:T_status'] [178.149719238, 10.0] {'PPMS:T_pm': 178.149719238, 'PPMS:T_status': 10.0}
1 0.003988981246948242
1 ['PPMS:H_pm', 'PPMS:H_status'] [nan, 1.0] {'PPMS:H_pm': nan, 'PPMS:H_status': 1.0}
2 0.006029844284057617
2 ['PPMS:SetField', 'PPMS:FieldRate', 'PPMS:FieldApproachMode', 'PPMS:MagnetMode'] [0.0, 10.0, 0.0, 0.0] {'PPMS:SetField': 0.0, 'PPMS:FieldRate': 10.0, 'PPMS:FieldApproachMode': 0.0, 'PPMS:MagnetMode': 0.0}
3 0.0010063648223876953
3 ['PPMS:SetTemp', 'PPMS:TempRate', 'PPMS:TempApproachMode'] [200.0, 0.5, 0.0] {'PPMS:SetTemp': 200.0, 'PPMS:TempRate': 0.5, 'PPMS:TempApproachMode': 0.0}
4 0.000993490219116211
4 ['PPMS:P_pm', 'PPMS:P_unit'] [7.81126499176, 1.0] {'PPMS:P_pm': 7.81126499176, 'PPMS:P_unit': 1.0}
5 0.05269765853881836
5 ['SR830-1:X', 'SR830-1:Y', 'SR830-1:R', 'SR830-1:Theta'] [1.14903e-06, -4.54022e-08, 1.14972e-06, -2.256] {'SR830-1:X': 1.14903e-06, 'SR830-1:Y': -4.54022e-08, 'SR830-1:R': 1.14972e-06, 'SR830-1:Theta': -

In [6]:
def ramp_back_to_0(total_step=0.2):
    switch_file('ramp_back_0')
    check_output = error_response(rp100.get_output)[0]
    
    v1,v2 = error_response(rp100.get_set_source_value)[0]
    v1,v2 = v1*int(check_output[0]), v2*int(check_output[1])
    error_response(rp100.set_source_value,
                          v1,v2)

    step_1, step_2 = total_step/(abs(v1)+abs(v2))*v1, total_step/(abs(v1)+abs(v2))*v2,
    step_num = int(v1/step_1)
    print('start_values',v1,v2)
    for _ in range(1):
        N = -step_num
        if N<0:
            step_1 = -step_1
            step_2 = -step_2
            N=-N
        for i in range(N):
            
            pause_process()
            time.sleep(1)
            error_response(rp100.set_source_value,
                           v1+(i+1)*step_1,
                           v2+step_2*(i+1))
            resume_process()
            
            time.sleep(1)
        time.sleep(0.5)
    error_response(rp100.set_source_value,
                           0,
                           0)
    print('Now safe to turn off the output by')
    print('error_response(rp100.set_output,0,0)')
# ramp_back_to_0(total_step=0.5)

In [7]:
def ramp_up(total_step=0.2):
    switch_file('ramp_back_0')
    check_output = error_response(rp100.get_output)[0]
    
    v1,v2 = error_response(rp100.get_set_source_value)[0]
    v1,v2 = v1*int(check_output[0]), v2*int(check_output[1])
    error_response(rp100.set_source_value,
                          v1,v2)

    step_1, step_2 = total_step/(abs(v1)+abs(v2))*v1, total_step/(abs(v1)+abs(v2))*v2,
    step_num = int(v1/step_1)
    print('start_values',v1,v2)
    for _ in range(1):
        step_1 = -0.05
        step_2 = 0.1
        N = -step_num
        if N<0:
            step_1 = -step_1
            step_2 = -step_2
            N=-N
        for i in range(N):
            
            pause_process()
            time.sleep(1)
            error_response(rp100.set_source_value,
                           v1+(i+1)*step_1,
                           v2+step_2*(i+1))
            resume_process()
            
            time.sleep(1)
        time.sleep(0.5)
    # error_response(rp100.set_source_value,
    #                        0,
    #                        0)
    print('Now safe to turn off the output by')
    print('error_response(rp100.set_output,0,0)')

In [23]:
ramp_back_to_0()

RP100 Initializaing resource manager
bingo!
start_values -14.999605 30.000059
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo

### Sample params and utility

In [8]:
dims = [200, 40, 50] #
C0=1.106

In [9]:
def get_disp(Cd,T,C0=None):
    def d0(T):
        return 2.31-0.00128*T-3.9e-5*T**2+1.1e-7*T**3-1.97e-10*T**4
    if C0 == None:
        return 53.123/(Cd-0.03768)-d0(T)-49.254
    else:
        return 53.123/(Cd-0.03768)-53.123/(C0-0.03768)

def get_strain(Cd,dims,T,C0=None,):
    f = get_stress(Cf,T,C0)
    return f/dims[0]/dims[1]*1e12/1e9

# def get_stress(Cf,T,C0=None):
#     def f0(T):
#         return 1059.79+123-0.0603*T-0.00233*T**2+7.84e-6*T**3-1.29e-8*T**4
    
#     if C0 == None:
#         return 1/(0.91+5e-5*T+9e-7*T**2)*(1582.75/(Cf-0.0839)-f0(T))
#     else:
#         return 1/(0.91+5e-5*T+9e-7*T**2)*(1582.75/(Cf-0.0839)-1582.75/(C0-0.0839))

# def get_pressure(Cf,dims,T,C0=None,):
#     f = get_stress(Cf,T,C0)
#     return f/dims[0]/dims[1]*1e12/1e9

In [10]:
# C0=1.111175
def read_all_quiet(funcs_to_call,read_col_names=read_col_names):
        all_data = []
        all_data_name = []
        for i in range(len(funcs_to_call)):
            t1 = time.time()
            data,name = error_response(funcs_to_call[i])
            # print(i,time.time()-t1)
            # print(i,name,data,{key:value for key,value in zip(name,data)})
            all_data += data
            all_data_name += name
        return np.array(all_data), np.array(all_data_name)
# def inquire_pressure(C0):
#     all_data, all_data_name = read_all_quiet(read_funcs)
#     read_dict = {key:value for key,value in zip(all_data_name,all_data,)}
#     return get_pressure(read_dict['AH:C_d'],dims,read_dict['PPMS:T_pm'],C0)
# def inquire_cernox():
#     all_data, all_data_name = read_all_quiet(read_funcs)
#     read_dict = {key:value for key,value in zip(all_data_name,all_data,)}
#     return cernox_temp(read_dict['SR830-2:R'])
# inquire_pressure(1.44)
# get_pressure(1.55,dims,145,1.4865)

## Measurement Sequences

### keep reading

In [24]:
cls_all(clear_funcs)
reset_all(allInstrumentList)
t0 = time.time()#wait until test
t1 = t0
start_process(  ncol=ncol,
                nrow=nrow,
                data_folder=data_folder,
                default_file='sweep_back',
                read_col_names=read_col_names,
                plot_axes_col_names=plot_axes_col_names,
                read_funcs=read_funcs,
                if_plot=if_plot,
                PID_control_map = PID_control_map,
                init_PID_value = init_PID_value,
                clear_funcs=clear_funcs,
                max_plot_N=100,
                start_time=t0
             )

SR830-1 Initializaing resource manager
SR830-2 Initializaing resource manager
KL6221 Initializaing resource manager


In [13]:
switch_file('Tsweep')
kl6221.set_ac_source(100e-6)

In [33]:
PID_paras = {
    'strain':{
        'new_target':1.11,
        'new_limits':[-20,100],
        'new_max_rate':None,
        'new_max_size':None,
        'new_multiplier':[-30,60]
    }
}
update_PID(PID_paras)

In [17]:
rp100.set_output(True,True)


In [22]:
switch_file('cd-170-2')

In [37]:
pm.set_field(30000,20,0,0)

'0'

In [19]:
stop_process()

### freq sweep at different temps

In [95]:
# freq sweep at temps
temps = np.concatenate([
    #     np.linspace(200,3e2,200),
    #     np.arange(300, 2, 1.5)
        np.array([100]),
    #     np.array([2])
     ]
    )
rate=2
read_csv_func = partial(read_max_idx,folder=data_folder)
# wait_until(read_csv_func,'sys:time','>',300,stablize=False)


#define parameters
if True:
    freqs = 10**(np.concatenate([
    #     np.linspace(200,3e2,200),
    #     np.arange(300, 2, 1.5)
        np.linspace(4,6,101),
    #     np.array([2])
     ]
    ))
#     rate = 0.5
#     current_values = np.interp(
#         final_Ts,
#         np.array([2,10,50,100,300]),
#         np.array([0.2,0.3,0.5,0.7,0.9])/1000
#     )

# kl6220.set_output(0)

for T in temps:

    cls_all(clear_funcs)
    reset_all(allInstrumentList)
    t0 = time.time()#wait until test
    t1 = t0
    start_process(ncol=ncol,
                  nrow=nrow,
                  data_folder=data_folder,
                  default_file=default_file,
                  read_col_names=read_col_names,
                  plot_axes_col_names=plot_axes_col_names,
                  read_funcs=read_funcs,
                 if_plot=if_plot,
                 clear_funcs=clear_funcs,
                 max_plot_N=100,
                 start_time=t0)
    
    pause_process() 
    time.sleep(0.5)
    start_T = error_response(pm.partial_get_data)[0][2]
    print('start_T',start_T)
    time.sleep(0.5)
    resume_process()
    #     final_T = 300 #K
    pause_process()
    time.sleep(20)
    switch_file(f'FeGe-accal-sample-freqsweep-1Hz-10kHz-MFLI-{T}K-2')
    pm.set_temp(T,rate,0)
    if start_T > T:
        comp_code = '<'
        temp = 1
    else:
        comp_code = '>'
        temp = -1
    time.sleep(0.5)
    resume_process()
    time.sleep(10)
    res = wait_until(read_csv_func,
                     entry_names=['PPMS:T_pm','PPMS:T_pm'],
                     comp_codes=['<',comp_code],
                     set_values=[0.001,T+temp*0.01],
                     stablizes=[True,False])
    # initialize thermal electric
    pause_process()
    time.sleep(0.5)
    
    time.sleep(10)
    resume_process()
    
    sweep_results = []
    # freq stablize
    for i in range(len(freqs)):
        
        time.sleep(0.5)
        
        pause_process()
        time.sleep(0.5)
        
    #     start_T = error_response(pm.partial_get_data)[0][2]
    #     print('start_T',start_T)
    #     time.sleep(0.5)
    #     resume_process()
        
        pause_process()
        time.sleep(0.5)
        error_response(Sr830_1.set_frequency,freqs[i])

        time.sleep(0.5)
        resume_process()
        time.sleep(120)
    #     res = wait_until(read_csv_func,
    #                      entry_names=['SR860-1:X'],
    #                      comp_codes=['<'],
    #                      set_values=[0.001],
    #                      stablizes=[True,False])
        
    #     print(res)
    #     sweep_results.append(res)
    #     time.sleep(60)
        if time.time()>t1+3600:
            stop_process()
            time.sleep(2)
            last_file = get_highest_indx_file(data_folder)[2]
            print('restarting...')
            reset_all(allInstrumentList)
            start_process(ncol=ncol,
                  nrow=nrow,
                  data_folder=data_folder,
                  default_file=last_file,
                  read_col_names=read_col_names,
                  plot_axes_col_names=plot_axes_col_names,
                  read_funcs=read_funcs,
                 if_plot=if_plot,
                 clear_funcs=clear_funcs,
                 max_plot_N=100,
                 start_time=t0)
            t1 = time.time()
            time.sleep(10)
    stop_process()

RP100 Initializaing resource manager
SR830-2 Initializaing resource manager
AH Initializaing resource manager
start_T 6.1433
SR830-1 Initializaing resource manager
restarting...
SR830-1 Initializaing resource manager
restarting...
SR830-1 Initializaing resource manager
1th trial failed Valid frequencies are between 0.001 Hz and 102 kHz.
2th trial failed Valid frequencies are between 0.001 Hz and 102 kHz.
3th trial failed Valid frequencies are between 0.001 Hz and 102 kHz.
4th trial failed Valid frequencies are between 0.001 Hz and 102 kHz.
5th trial failed Valid frequencies are between 0.001 Hz and 102 kHz.
6th trial failed Valid frequencies are between 0.001 Hz and 102 kHz.
7th trial failed Valid frequencies are between 0.001 Hz and 102 kHz.
8th trial failed Valid frequencies are between 0.001 Hz and 102 kHz.
9th trial failed Valid frequencies are between 0.001 Hz and 102 kHz.
10th trial failed Valid frequencies are between 0.001 Hz and 102 kHz.
11th trial failed Valid frequencies are

In [11]:
stop_process()

### frequency sweep

In [11]:
# freq sweep
read_csv_func = partial(read_max_idx,folder=data_folder)
# wait_until(read_csv_func,'sys:time','>',300,stablize=False)


t0 = time.time()#wait until test
t1 = t0
cls_all(clear_funcs)
reset_all(allInstrumentList)
start_process(ncol=ncol,
              nrow=nrow,
              data_folder=data_folder,
              default_file=default_file,
              read_col_names=read_col_names,
              plot_axes_col_names=plot_axes_col_names,
              read_funcs=read_funcs,
             if_plot=if_plot,
             clear_funcs=clear_funcs,
             max_plot_N=100,
             start_time=t0)

# initialize thermal electric

# # set up power
# power = 1
# power_steps_N = 10
# for i in range(power_steps_N+1):
#     pause_process()
#     error_response(Sr830_1.set_amplitude,power/power_steps_N*i)
#     time.sleep(1)
#     resume_process()
    
# pause_process()
# time.sleep(1)
# error_response(Sr860_1.auto_range)
# error_response(Sr860_1.auto_scale)
# # error_response(Sr860_1.auto_scale())
# time.sleep(1)
# resume_process()

RP100 Initializaing resource manager
SR830-1 Initializaing resource manager
SR830-2 Initializaing resource manager
AH Initializaing resource manager
KL6221 Initializaing resource manager


In [ ]:
# t0 = time.time()#wait until test
# t1 = t0
pause_process()
time.sleep(0.5)
switch_file('LCR-acECE-sample-freqsweep-1Hz-100Hz-SR830-10K-without554')
time.sleep(10)
resume_process()

# error_response(kl6221.set_ac_source,10e-5)
# error_response(kl6221.set_ac_output,1)
#define parameters
if True:
    freqs = 10**(np.concatenate([
    #     np.linspace(200,3e2,200),
    #     np.arange(300, 2, 1.5)
        # np.linspace(-0.0013,3.0235,100),
        np.linspace(-0.0234,2.0234,41),
    #     np.array([2])
     ]
    ))
#     rate = 0.5
#     current_values = np.interp(
#         final_Ts,
#         np.array([2,10,50,100,300]),
#         np.array([0.2,0.3,0.5,0.7,0.9])/1000
#     )

# kl6220.set_output(0)


sweep_results = []
# freq stablize
for i in range(len(freqs)):
    time.sleep(0.5)
    
    pause_process()
    time.sleep(0.5)
    
#     start_T = error_response(pm.partial_get_data)[0][2]
#     print('start_T',start_T)
#     time.sleep(0.5)
#     resume_process()
    
    pause_process()
    time.sleep(0.5)
    error_response(kl6221.set_ac_freq,freqs[i])
#     # set_current
#     kl6220.set_source_value(current_values[i])
    
    
#     if start_T > final_Ts[i]:
#         comp_code = '<'
#         temp = 1
#     else:
#         comp_code = '>'
#         temp = -1
    time.sleep(0.5)
    resume_process()
    time.sleep(300)
#     res = wait_until(read_csv_func,
#                      entry_names=['SR860-1:X'],
#                      comp_codes=['<'],
#                      set_values=[0.001],
#                      stablizes=[True,False])
    
#     print(res)
#     sweep_results.append(res)
#     time.sleep(60)
    if time.time()>t1+3600:
        stop_process()
        time.sleep(2)
        last_file = get_highest_indx_file(data_folder)[2]
        print('restarting...')
        reset_all(allInstrumentList)
        start_process(ncol=ncol,
              nrow=nrow,
              data_folder=data_folder,
              default_file=last_file,
              read_col_names=read_col_names,
              plot_axes_col_names=plot_axes_col_names,
              read_funcs=read_funcs,
             if_plot=if_plot,
             clear_funcs=clear_funcs,
             max_plot_N=100,
             start_time=t0)
        t1 = time.time()
        time.sleep(10)
# kl6220.set_output(0)

KL6221 Initializaing resource manager
restarting...
KL6221 Initializaing resource manager


In [17]:
clear_plot()

In [30]:
stop_process()

### power sweep

In [12]:

#     )


t0 = time.time()#wait until test
t1 = t0
start_process(ncol=ncol,
              nrow=nrow,
              data_folder=data_folder,
              default_file=default_file,
              read_col_names=read_col_names,
              plot_axes_col_names=plot_axes_col_names,
              read_funcs=read_funcs,
             if_plot=if_plot,
             clear_funcs=clear_funcs,
             max_plot_N=100,
             start_time=t0)

# initialize thermal electric
pause_process()
time.sleep(0.5)
switch_file('psweep')
time.sleep(10)
resume_process()

# # set up power
# power = 1
# power_steps_N = 10
# for i in range(power_steps_N+1):
#     pause_process()
#     error_response(Sr830_1.set_amplitude,power/power_steps_N*i)
#     time.sleep(1)
#     resume_process()
    
# pause_process()
# time.sleep(1)
# error_response(Sr860_1.auto_range)
# error_response(Sr860_1.auto_scale)
# # error_response(Sr860_1.auto_scale())
# time.sleep(1)
# resume_process()

In [12]:
# power sweep


read_csv_func = partial(read_max_idx,folder=data_folder)
# wait_until(read_csv_func,'sys:time','>',300,stablize=False)
# kl6221.set_ac_freq(0.313)
cls_all(clear_funcs)
reset_all(allInstrumentList)
#define parameters
if True:
    powers = (np.concatenate([
    #     np.linspace(200,3e2,200),
    #     np.arange(300, 2, 1.5)
        np.linspace(20e-6,200e-6,10),
    #     np.array([2])
     ]
    ))
#     rate = 0.5
#     current_values = np.interp(
#         final_Ts,
#         np.array([2,10,50,100,300]),
#         np.array([0.2,0.3,0.5,0.7,0.9])/1000
switch_file('psweep')
sweep_results = []
# freq stablize
# freq = 164.234
# error_response(kl6221.set_frequency,freq)
for i in range(len(powers)):
    
    time.sleep(0.5)
    
    pause_process()
    
    time.sleep(0.5)
    
#     start_T = error_response(pm.partial_get_data)[0][2]
#     print('start_T',start_T)
#     time.sleep(0.5)
#     resume_process()
    
    pause_process()
    time.sleep(0.5)
    error_response(kl6221.set_ac_source,powers[i])
#     # set_current
#     kl6220.set_source_value(current_values[i])
    
    
#     if start_T > final_Ts[i]:
#         comp_code = '<'
#         temp = 1
#     else:
#         comp_code = '>'
#         temp = -1
    time.sleep(0.5)
    resume_process()
    for i in range(5):
        time.sleep(1)
#     res = wait_until(read_csv_func,
#                      entry_names=['SR860-1:X'],
#                      comp_codes=['<'],
#                      set_values=[0.001],
#                      stablizes=[True,False])
    
#     print(res)
#     sweep_results.append(res)
#     time.sleep(60)
    if time.time()>t1+3600:
        stop_process()
        time.sleep(2)
        last_file = get_highest_indx_file(data_folder)[2]
        print('restarting...')
        reset_all(allInstrumentList)
        start_process(ncol=ncol,
              nrow=nrow,
              data_folder=data_folder,
              default_file=last_file,
              read_col_names=read_col_names,
              plot_axes_col_names=plot_axes_col_names,
              read_funcs=read_funcs,
             if_plot=if_plot,
             clear_funcs=clear_funcs,
             max_plot_N=100,
             start_time=t0)
        t1 = time.time()
        time.sleep(10)
# kl6220.set_output(0)

RP100 Initializaing resource manager
SR830-1 Initializaing resource manager
SR830-2 Initializaing resource manager
AH Initializaing resource manager
KL6221 Initializaing resource manager
KL6221 Initializaing resource manager


In [19]:
stop_process()

### temp sweep from xx to xx

In [ ]:
# temp sweep


read_csv_func = partial(read_max_idx,folder=data_folder)
# wait_until(read_csv_func,'sys:time','>',300,stablize=False)


#define parameters
if True:
    final_Ts = np.concatenate([
        np.array([1.8]),
        # np.linspace(1.8,1.8,1),
        # np.linspace(120,140,51),
        # np.linspace(140,300,51)
    #     np.arange(300, 2, 1.5)
        # np.linspace(100,80,101),
        # np.linspace(80,50,61),
        # np.linspace(50,20,151),
        # np.array([150,10]),
        # np.array([150]),
        # np.array([10,300]),
     ]
    )
    rate = 0.65
# last_file = get_highest_indx_file(data_folder)[2]
# kl6220.set_output(0)
# error_response(Sr830_1.set_frequency,162.234)
# last_file = get_highest_indx_file(data_folder)[2]
t0 = time.time()#wait until test
t1 = t0
cls_all(clear_funcs)
reset_all(allInstrumentList)
start_process(ncol=ncol,
              nrow=nrow,
              data_folder=data_folder,
              default_file=default_file,
              read_col_names=read_col_names,
              plot_axes_col_names=plot_axes_col_names,
              read_funcs=read_funcs,
             if_plot=if_plot,
             clear_funcs=clear_funcs,
             max_plot_N=100,
             start_time=t0)

# initialize thermal electric
pause_process()
time.sleep(0.5)
# switch_file('FeGe-Tsweep-7-7')
time.sleep(10)
resume_process()

sweep_results = []
# temperature stablize
for i in range(len(final_Ts)):
    
    time.sleep(0.5)
    
    pause_process()
    time.sleep(0.5)
    
    start_T = error_response(pm.partial_get_data)[0][2]
    print('start_T',start_T)
    time.sleep(0.5)
    resume_process()
    
#     final_T = 300 #K
    pause_process()
    time.sleep(0.5)
    pm.set_temp(final_Ts[i],rate,0)
    
#     # set_current
#     kl6220.set_source_value(current_values[i])
    
    
    if start_T > final_Ts[i]:
        comp_code = '<'
        temp = 1
    else:
        comp_code = '>'
        temp = -1
    time.sleep(0.5)
    resume_process()
    time.sleep(10)
    res = wait_until(read_csv_func,
                     entry_names=['PPMS:T_pm','PPMS:T_pm'],
                     comp_codes=['<',comp_code],
                     set_values=[0.001,final_Ts[i]+temp*0.01],
                     stablizes=[True,False])
    
#     print(res)
    if i == 0:
        time.sleep(0)
    sweep_results.append(res)
    time.sleep(500)
    if time.time()>t1+3600:
        stop_process()
        time.sleep(2)
        last_file = get_highest_indx_file(data_folder)[2]
        cls_all(clear_funcs)
        reset_all(allInstrumentList)
        print('restarting...')
        start_process(ncol=ncol,
              nrow=nrow,
              data_folder=data_folder,
              default_file=last_file,
              read_col_names=read_col_names,
              plot_axes_col_names=plot_axes_col_names,
              read_funcs=read_funcs,
             if_plot=if_plot,
             clear_funcs=clear_funcs,
             max_plot_N=100,
             start_time=t0)
        t1 = time.time()
        time.sleep(10)
# kl6220.set_output(0)


RP100 Initializaing resource manager
SR830-1 Initializaing resource manager
SR830-2 Initializaing resource manager
AH Initializaing resource manager
KL6221 Initializaing resource manager
start_T 39.7435
start_T 38.0008
start_T 39.6412
start_T 41.2807
start_T 42.9209
start_T 44.56
RP100 Initializaing resource manager
SR830-1 Initializaing resource manager
SR830-2 Initializaing resource manager
AH Initializaing resource manager
KL6221 Initializaing resource manager
restarting...
start_T 46.2002
start_T 47.8402
start_T 49.4799
start_T 51.1197
start_T 52.7589
start_T 54.3996
RP100 Initializaing resource manager
SR830-1 Initializaing resource manager
SR830-2 Initializaing resource manager
AH Initializaing resource manager
KL6221 Initializaing resource manager
restarting...
start_T 56.0372
start_T 57.6784
start_T 59.3206
start_T 60.9576
start_T 62.6006
start_T 64.2397
RP100 Initializaing resource manager
SR830-1 Initializaing resource manager
SR830-2 Initializaing resource manager
AH Initial

In [ ]:
stop_process()

In [ ]:
# temp sweep


read_csv_func = partial(read_max_idx,folder=data_folder)
# wait_until(read_csv_func,'sys:time','>',300,stablize=False)


#define parameters
if True:
    final_Ts = np.concatenate([
        np.linspace(150,70,401),
    #     np.arange(300, 2, 1.5)
        # np.linspace(100,80,101),
        # np.linspace(80,50,61),
        # np.linspace(50,20,151),
        # np.array([150,10]),
        # np.array([150]),
        # np.array([10,300]),
     ]
    )
    rate = 2

# kl6220.set_output(0)
# error_response(Sr830_1.set_frequency,162.234)
t0 = time.time()#wait until test
t1 = t0
cls_all(clear_funcs)
reset_all(allInstrumentList)
start_process(ncol=ncol,
              nrow=nrow,
              data_folder=data_folder,
              default_file=default_file,
              read_col_names=read_col_names,
              plot_axes_col_names=plot_axes_col_names,
              read_funcs=read_funcs,
             if_plot=if_plot,
             clear_funcs=clear_funcs,
             max_plot_N=100,
             start_time=t0)

# initialize thermal electric
pause_process()
time.sleep(0.5)
switch_file('FeGe-Tsweep-10-5')
time.sleep(10)
resume_process()

sweep_results = []
# temperature stablize
for i in range(len(final_Ts)):
    
    time.sleep(0.5)
    
    pause_process()
    time.sleep(0.5)
    
    start_T = error_response(pm.partial_get_data)[0][2]
    print('start_T',start_T)
    time.sleep(0.5)
    resume_process()
    
#     final_T = 300 #K
    pause_process()
    time.sleep(0.5)
    pm.set_temp(final_Ts[i],rate,0)
    
#     # set_current
#     kl6220.set_source_value(current_values[i])
    
    
    if start_T > final_Ts[i]:
        comp_code = '<'
        temp = 1
    else:
        comp_code = '>'
        temp = -1
    time.sleep(0.5)
    resume_process()
    time.sleep(10)
    res = wait_until(read_csv_func,
                     entry_names=['PPMS:T_pm','PPMS:T_pm'],
                     comp_codes=['<',comp_code],
                     set_values=[0.001,final_Ts[i]+temp*0.01],
                     stablizes=[True,False])
    
#     print(res)
    if i == 0:
        time.sleep(300)
    sweep_results.append(res)
    time.sleep(180)
    if time.time()>t1+3600:
        stop_process()
        time.sleep(2)
        last_file = get_highest_indx_file(data_folder)[2]
        cls_all(clear_funcs)
        reset_all(allInstrumentList)
        print('restarting...')
        start_process(ncol=ncol,
              nrow=nrow,
              data_folder=data_folder,
              default_file=last_file,
              read_col_names=read_col_names,
              plot_axes_col_names=plot_axes_col_names,
              read_funcs=read_funcs,
             if_plot=if_plot,
             clear_funcs=clear_funcs,
             max_plot_N=100,
             start_time=t0)
        t1 = time.time()
        time.sleep(10)
# kl6220.set_output(0)


RP100 Initializaing resource manager
SR830-1 Initializaing resource manager
SR830-2 Initializaing resource manager
AH Initializaing resource manager
KL6221 Initializaing resource manager
start_T 150.0013
start_T 150.0006
start_T 149.8019
start_T 149.5988
start_T 149.4011
start_T 149.2008
start_T 148.9999
start_T 148.8005
start_T 148.6004
start_T 148.4016
RP100 Initializaing resource manager
SR830-1 Initializaing resource manager
SR830-2 Initializaing resource manager
AH Initializaing resource manager
KL6221 Initializaing resource manager
restarting...
start_T 148.2014
start_T 148.0029
start_T 147.8005
start_T 147.6013
start_T 147.4015
start_T 147.2022
start_T 147.0015
start_T 146.8023
start_T 146.6036
start_T 146.4017
start_T 146.2038
RP100 Initializaing resource manager
SR830-1 Initializaing resource manager
SR830-2 Initializaing resource manager
AH Initializaing resource manager
KL6221 Initializaing resource manager
restarting...
start_T 146.003
start_T 145.8036
start_T 145.598
start

In [ ]:
stop_process()

### temp sweep at different fields

In [21]:
# temp sweep


read_csv_func = partial(read_max_idx,folder=data_folder)
# wait_until(read_csv_func,'sys:time','>',300,stablize=False)


#define parameters
if True:
    final_Ts = np.concatenate([
    #     np.linspace(200,3e2,200),
    #     np.arange(300, 2, 1.5)
        np.linspace(34,38,41),
    #     np.array([2])
     ]
    )
    fields = np.concatenate([
    #     np.linspace(200,3e2,200),
    #     np.arange(300, 2, 1.5)
        # np.linspace(35,37,21),
        np.array([5000])
        
     ]
    )
    rate = 0.5

# kl6220.set_output(0)
for field in fields:
    
    t0 = time.time()#wait until test
    t1 = t0
    cls_all(clear_funcs)
    reset_all(allInstrumentList)
    start_process(ncol=ncol,
                  nrow=nrow,
                  data_folder=data_folder,
                  default_file=default_file,
                  read_col_names=read_col_names,
                  plot_axes_col_names=plot_axes_col_names,
                  read_funcs=read_funcs,
                 if_plot=if_plot,
                 clear_funcs=clear_funcs,
                 max_plot_N=100,
                 start_time=t0)
    # final_H = finalHs[i] #Oe
    pause_process()
    time.sleep(0.5)
    start_H = error_response(pm.partial_get_data)[0][3]
#     switch_file('unknown_rampH%.2f-%.2f'%(start_H,final_H))
    print('from {} to {}'.format(start_H,field))
    switch_file(f'TbMn6Sn6-accal-sample-34-38K_9.479Hz_100V_-100V_{field}Oe')
    time.sleep(0.5)
    resume_process()

    final_H=field
    pause_process()
    time.sleep(0.5)
    print('start setting')
    if start_H > final_H:
        comp_code = '<'
        temp = 1
    else:
        comp_code = '>'
        temp = -1
    error_response(pm.set_field,final_H,rate,0,0,cls_func=pm.cls_ppms)
    time.sleep(5)

    if abs(start_H - final_H)>10:
        print('H set to %.2f'%final_H)
    else:
        print('no operation')
        
#     time.sleep(0.5)
    resume_process()
#     time.sleep(int(abs(final_H-start_H)/rate))       
    time.sleep(10)
    res = wait_until(read_csv_func,
                     entry_names=['PPMS:H_pm','PPMS:H_pm'],
                     comp_codes=['<',comp_code],
                     set_values=[0.01,final_H+temp*10],
                     stablizes=[True,False])
    
    sweep_results = []
    # temperature stablize
    for i in range(len(final_Ts)):
        
        time.sleep(0.5)
        
        pause_process()
        time.sleep(0.5)
        
        start_T = error_response(pm.partial_get_data)[0][2]
        print('start_T',start_T)
        time.sleep(0.5)
        resume_process()
        
    #     final_T = 300 #K
        pause_process()
        time.sleep(0.5)
        pm.set_temp(final_Ts[i],rate,0)
        
    #     # set_current
    #     kl6220.set_source_value(current_values[i])
        
        
        if start_T > final_Ts[i]:
            comp_code = '<'
            temp = 1
        else:
            comp_code = '>'
            temp = -1
        time.sleep(0.5)
        resume_process()
        time.sleep(10)
        res = wait_until(read_csv_func,
                         entry_names=['PPMS:T_pm','PPMS:T_pm'],
                         comp_codes=['<',comp_code],
                         set_values=[0.001,final_Ts[i]+temp*0.01],
                         stablizes=[True,False])
        if i==0:
            time.sleep(1200)
    #     print(res)
        sweep_results.append(res)
        time.sleep(600)
        if time.time()>t1+3600:
            stop_process()
            time.sleep(2)
            last_file = get_highest_indx_file(data_folder)[2]
            cls_all(clear_funcs)
            reset_all(allInstrumentList)
            print('restarting...')
            start_process(ncol=ncol,
                  nrow=nrow,
                  data_folder=data_folder,
                  default_file=last_file,
                  read_col_names=read_col_names,
                  plot_axes_col_names=plot_axes_col_names,
                  read_funcs=read_funcs,
                 if_plot=if_plot,
                 clear_funcs=clear_funcs,
                 max_plot_N=100,
                 start_time=t0)
            t1 = time.time()
            time.sleep(10)
# kl6220.set_output(0)


RP100 Initializaing resource manager
SR830-1 Initializaing resource manager
AH Initializaing resource manager
PPMS Initializaing resource manager
from -0.29 to 5000
start setting
H set to 5000.00
start_T 36.9999


KeyboardInterrupt: 

In [81]:
stop_process()

### Field sweep

In [6]:
t0 = time.time()#wait until test
t1 = t0
T_start = error_response(pm.partial_get_data)[0][2]
current_value = np.interp(
    T_start,
    np.array([2,10,50,100,300]),
    np.array([0.2,0.3,0.5,0.7,0.9])/1000
)
kl6220.set_source_value(0.3e-3)
time.sleep(0.5)
kl6220.set_output(1)
time.sleep(0.5)
# stop_process()
start_process(ncol=ncol,
              nrow=nrow,
              data_folder=data_folder,
              default_file=default_file,
              read_col_names=read_col_names,
              plot_axes_col_names=plot_axes_col_names,
              read_funcs=read_funcs,
             if_plot=if_plot,
             clear_funcs=clear_funcs,
             max_plot_N=100,
             start_time=t0)

In [10]:
#sweep field wait until
read_csv_func = partial(read_max_idx,folder=data_folder)
# res = wait_until(read_csv_func,
#                      entry_names=['PPMS:T_pm','PPMS:T_pm','KL2182A1:V'],
#                      comp_codes=['<',comp_code,'<'],
#                      set_values=[0.001,2+temp*0.01,0.1e-7],
#                      stablizes=[True,False,True])
# end_2=120
max_errors = 10
finalHs = np.concatenate([
                    np.array([9e4]),
#                     np.linspace(0,9e4,101),
                    np.linspace(9e4,-9e4,201),
                    np.linspace(-9e4,90000,201),
#                     np.linspace(9e4,0,101),
                    np.array([0]),
])
rate = 80
results = []
switch_file('EJC-022-EuGe_S3_j-along-a_B-along-b_TE_ChrCon_01_rampH')
t0 = time.time()#wait until test
t1 = t0
time.sleep(30)
for i in range(len(finalHs)):
#     time.sleep(0.5)
    final_H = finalHs[i] #Oe
    pause_process()
    time.sleep(0.5)
    kl6220.set_source_value(0.2e-3)
    time.sleep(0.5)
    kl6220.set_output(1)
    time.sleep(0.5)
    start_H = error_response(pm.partial_get_data)[0][3]
#     switch_file('unknown_rampH%.2f-%.2f'%(start_H,final_H))
    print('start_H',start_H)
    time.sleep(0.5)
    resume_process()
    
    
    pause_process()
    time.sleep(0.5)
    print('start setting')
    if start_H > final_H:
        comp_code = '<'
        temp = 1
    else:
        comp_code = '>'
        temp = -1
    error_response(pm.set_field,final_H,rate,0,0,cls_func=pm.cls_ppms)
    time.sleep(5)

    if abs(start_H - final_H)>10:
        print('H set to %.2f'%final_H)
    else:
        print('no operation')
        
#     time.sleep(0.5)
    resume_process()
#     time.sleep(int(abs(final_H-start_H)/rate))       
    time.sleep(10)
    res = wait_until(read_csv_func,
                     entry_names=['PPMS:H_pm','PPMS:H_pm','KL2182A1:V'],
                     comp_codes=['<',comp_code,''],
                     set_values=[0.01,final_H+temp*10,0.1e-7],
                     stablizes=[True,False,True])
    results.append(res)
    if time.time()>t1+3600:
        stop_process()
        time.sleep(2)
        last_file = get_highest_indx_file(data_folder)[2]
        print('restarting...')
        start_process(ncol=ncol,
              nrow=nrow,
              data_folder=data_folder,
              default_file=last_file,
              read_col_names=read_col_names,
              plot_axes_col_names=plot_axes_col_names,
              read_funcs=read_funcs,
             if_plot=if_plot,
             clear_funcs=clear_funcs,
             max_plot_N=100,
             start_time=t0)
        t1 = time.time()
        time.sleep(10)
#     time.sleep(60)

start_H 90000.289
start setting
no operation
start_H 90000.508
start setting
no operation
start_H 90000.383
start setting
H set to 89100.00
start_H 89100.25
start setting
H set to 88200.00
start_H 88200.367
start setting
H set to 87300.00
start_H 87300.281
start setting
H set to 86400.00
start_H 86400.336
start setting
H set to 85500.00
start_H 85500.375
start setting
H set to 84600.00
start_H 84600.305
start setting
H set to 83700.00
start_H 83700.195
start setting
H set to 82800.00
start_H 82800.211
start setting
H set to 81900.00
start_H 81900.352
start setting
H set to 81000.00
1th trial failed
start_H 81000.328
start setting
H set to 80100.00
start_H 80100.047
start setting
H set to 79200.00
start_H 79200.281
start setting
H set to 78300.00
start_H 78300.313
start setting
H set to 77400.00
start_H 77400.289
start setting
H set to 76500.00
start_H 76500.305
start setting
H set to 75600.00
start_H 75600.172
start setting
H set to 74700.00
start_H 74700.25
start setting
H set to 7380

H set to -54000.00
start_H -54000.117
start setting
H set to -54900.00
start_H -54900.16
start setting
H set to -55800.00
start_H -55890.281
start setting
H set to -56700.00
start_H -56700.191
start setting
H set to -57600.00
start_H -57600.238
start setting
H set to -58500.00
start_H -58500.246
start setting
H set to -59400.00
start_H -59400.313
start setting
H set to -60300.00
start_H -60469.316
start setting
H set to -61200.00
start_H -61200.367
start setting
H set to -62100.00
start_H -62100.367
start setting
H set to -63000.00
start_H -63000.188
start setting
H set to -63900.00
start_H -63900.34
start setting
H set to -64800.00
start_H -64800.238
start setting
H set to -65700.00
start_H -65700.266
start setting
H set to -66600.00
start_H -66600.164
start setting
H set to -67500.00
start_H -67500.313
start setting
H set to -68400.00
start_H -68400.305
start setting
H set to -69300.00
start_H -69510.258
start setting
H set to -70200.00
start_H -70200.242
start setting
H set to -7110

start_H 16200.388
start setting
H set to 17100.00
start_H 17100.352
start setting
H set to 18000.00
start_H 18000.291
start setting
H set to 18900.00
start_H 18900.402
start setting
H set to 19800.00
start_H 19800.217
start setting
H set to 20700.00
start_H 20700.201
start setting
H set to 21600.00
start_H 21600.27
start setting
H set to 22500.00
start_H 22500.316
start setting
H set to 23400.00
start_H 23400.324
start setting
H set to 24300.00
start_H 24300.158
start setting
H set to 25200.00
start_H 25200.227
start setting
H set to 26100.00
start_H 26100.338
start setting
H set to 27000.00
start_H 27000.32
start setting
H set to 27900.00
start_H 27900.305
start setting
H set to 28800.00
start_H 28800.502
start setting
H set to 29700.00
start_H 29700.527
start setting
H set to 30600.00
start_H 30600.406
start setting
H set to 31500.00
start_H 31500.221
start setting
H set to 32400.00
start_H 32400.309
start setting
H set to 33300.00
start_H 33300.188
start setting
H set to 34200.00
st

In [21]:
last_index = get_highest_indx_file(data_folder)[0]

In [22]:
last_index

316

In [12]:
len(results)

404

In [19]:
merged_df = pd.concat(results,axis=1,ignore_index=True).T

In [24]:
import os
merged_df.to_csv(os.path.join(data_folder,'stable-%03d-rampH.dat'%last_index),sep='\t',index=False)

### strain sweep

In [11]:
cls_all(clear_funcs)
reset_all(allInstrumentList)
start_process(ncol=ncol,
              nrow=nrow,
              data_folder=data_folder,
              default_file=default_file,
              read_col_names=read_col_names,
              plot_axes_col_names=plot_axes_col_names,
              read_funcs=read_funcs,
             if_plot=if_plot,
             clear_funcs=clear_funcs,
             max_plot_N=100,)


RP100 Initializaing resource manager
SR830-1 Initializaing resource manager
SR830-2 Initializaing resource manager
AH Initializaing resource manager
KL6221 Initializaing resource manager


In [25]:
rp100.get_set_source_value()

([-5.0003595, 9.9995089], ['RP100:setVolt1', 'RP100:setVolt2'])

In [12]:
switch_file('LCR_rampe_bfECE')

In [15]:
rp100.set_output(False,False)

In [19]:
# #sweep voltage increase
# time.sleep(0.5)
# pause_process()
# freq = 162.234
# error_response(Sr830_1.set_frequency,freq)
# # cls_all(clear_funcs)
# # reset_all(allInstrumentList)
# # rp100.set_output(True,True)
# resume_process()
# # time.sleep(600)

In [18]:
switch_file('strainsweep-40-30')
rp100.set_output(True,True)
for _ in range(1):
    step_1 = -0.05
    step_2 = 0.1
    N = -100

    
    # step_1 = -0.1
    # step_2 = 0.1
    # N = -70
     
    time.sleep(0.5)
    
    pause_process()
    time.sleep(0.5)
    start_1,start_2 = error_response(rp100.get_set_source_value)[0]

    
    print('start_values',start_1,start_2)
    time.sleep(0.5)
    resume_process()
    
    
    
#     pause_process()
#     time.sleep(0.5)
#     rp100.cls()
#     time.sleep(0.5)
#     error_response(rp100.set_output,True,True)
#     resume_process()
    time.sleep(0.5)
    if N<0:
        step_1 = -step_1
        step_2 = -step_2
        N=-N
    for i in range(N):
        
        pause_process()
        time.sleep(2)
        error_response(rp100.set_source_value,
                       start_1+(i+1)*step_1,
                       start_2+step_2*(i+1))
        resume_process()
        
        time.sleep(4)
    time.sleep(0.5)
    
    time.sleep(2)
    end_1,end_2 = error_response(rp100.get_set_source_value)[0]
    print('current_values',end_1,end_2)
    stress = error_response(get_strain,C0)
    print('current_strain',stress)
    time.sleep(0.5)
#     print(error_response())
    
    resume_process()

RP100 Initializaing resource manager
start_values -19.999228 40.000336
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
1th trial failed VI_ERROR_RSRC_BUSY (-1073807246): The resource is valid, but VISA cannot currently access it.
2th trial failed VI_ERROR_RSRC_BUSY (-1073807246): The resource is valid, but VISA cannot currently access it.
3th tri

In [20]:
rp100.set_output(True,True)
for _ in range(1):
    step_1 = -0.05
    step_2 = 0.1
    N = -500

    
    # step_1 = -0.1
    # step_2 = 0.1
    # N = -70
     
    time.sleep(0.5)
    
    pause_process()
    time.sleep(0.5)
    start_1,start_2 = error_response(rp100.get_set_source_value)[0]

    
    print('start_values',start_1,start_2)
    time.sleep(0.5)
    resume_process()
    
    
    
#     pause_process()
#     time.sleep(0.5)
#     rp100.cls()
#     time.sleep(0.5)
#     error_response(rp100.set_output,True,True)
#     resume_process()
    time.sleep(0.5)
    if N<0:
        step_1 = -step_1
        step_2 = -step_2
        N=-N
    for i in range(N):
        
        pause_process()
        time.sleep(2)
        error_response(rp100.set_source_value,
                       start_1+(i+1)*step_1,
                       start_2+step_2*(i+1))
        resume_process()
        
        time.sleep(4)
    time.sleep(0.5)
    
    time.sleep(2)
    end_1,end_2 = error_response(rp100.get_set_source_value)[0]
    print('current_values',end_1,end_2)
    # strain = error_response(get_current_stress,ah.trigger_and_read,C0,area)
    # print('current_stress',strain)
    time.sleep(0.5)
#     print(error_response())
    
    resume_process()

start_values -30.000172 59.9991
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bi

In [17]:
stop_process()

### temp sweep at different strains

In [10]:
# temp sweep


read_csv_func = partial(read_max_idx,folder=data_folder)
#define parameters
if True:
    final_Ts = np.concatenate([
    #     np.linspace(200,3e2,200),
    #     np.arange(300, 2, 1.5)
        np.linspace(34,38,41),
    #     np.array([2])
     ]
    )


for _ in range(1):
    step_1 = 0.5
    step_2 = -0.5
    N = -200
    cls_all(clear_funcs)
    reset_all(allInstrumentList)
    start_process(ncol=ncol,
                  nrow=nrow,
                  data_folder=data_folder,
                  default_file=default_file,
                  read_col_names=read_col_names,
                  plot_axes_col_names=plot_axes_col_names,
                  read_funcs=read_funcs,
                 if_plot=if_plot,
                 clear_funcs=clear_funcs,
                 max_plot_N=100,
                 start_time=t0)
    
    pause_process()
    time.sleep(0.5)
    start_1,start_2 = error_response(rp100.get_set_source_value)[0]

    switch_file('TbMn6Sn6_S2_1stcd_accal_rampe%.2f_%.2f-%.2f_%.2f'%(start_1,
                                                                    start_2,
                                                                    start_1+step_1*N,
                                                                    start_2+step_2*N))
    print('start_values',start_1,start_2)
    time.sleep(0.5)
    resume_process()
    
    
    
#     pause_process()
#     time.sleep(0.5)
#     rp100.cls()
#     time.sleep(0.5)
#     error_response(rp100.set_output,True,True)
#     resume_process()
    time.sleep(0.5)
    if N<0:
        step_1 = -step_1
        step_2 = -step_2
        N=-N
    for i in range(N):
        
        pause_process()
        time.sleep(2)
        error_response(rp100.set_source_value,
                       start_1+(i+1)*step_1,
                       start_2+step_2*(i+1))
        resume_process()
        
        time.sleep(4)
    time.sleep(0.5)
    
    time.sleep(0.5)
    end_1,end_2 = error_response(rp100.get_set_source_value)[0]
    print('current_values',end_1,end_2)
    strain = error_response(get_current_strain,ah.trigger_and_read,Cd0,L)
    print('current_strain',strain*100,'%')
    time.sleep(0.5)
#     print(error_response())

    sweep_results = []
    # temperature stablize
    for i in range(len(final_Ts)):
        
        time.sleep(0.5)
        
        pause_process()
        time.sleep(0.5)
        
        start_T = error_response(pm.partial_get_data)[0][2]
        print('start_T',start_T)
        time.sleep(0.5)
        resume_process()
        
    #     final_T = 300 #K
        pause_process()
        time.sleep(0.5)
        pm.set_temp(final_Ts[i],rate,0)
        
    #     # set_current
    #     kl6220.set_source_value(current_values[i])
        
        
        if start_T > final_Ts[i]:
            comp_code = '<'
            temp = 1
        else:
            comp_code = '>'
            temp = -1
        time.sleep(0.5)
        resume_process()
        time.sleep(10)
        res = wait_until(read_csv_func,
                         entry_names=['PPMS:T_pm','PPMS:T_pm'],
                         comp_codes=['<',comp_code],
                         set_values=[0.001,final_Ts[i]+temp*0.01],
                         stablizes=[True,False])
        if i==0:
            time.sleep(1200)
    #     print(res)
        sweep_results.append(res)
        time.sleep(600)
        if time.time()>t1+3600:
            stop_process()
            time.sleep(2)
            last_file = get_highest_indx_file(data_folder)[2]
            cls_all(clear_funcs)
            reset_all(allInstrumentList)
            print('restarting...')
            start_process(ncol=ncol,
                  nrow=nrow,
                  data_folder=data_folder,
                  default_file=last_file,
                  read_col_names=read_col_names,
                  plot_axes_col_names=plot_axes_col_names,
                  read_funcs=read_funcs,
                 if_plot=if_plot,
                 clear_funcs=clear_funcs,
                 max_plot_N=100,
                 start_time=t0)
            t1 = time.time()
            time.sleep(10)
    resume_process()




RP100 Initializaing resource manager
SR830-1 Initializaing resource manager
AH Initializaing resource manager
RP100 Initializaing resource manager
start_values 104.00047 -103.99986
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
bingo!
1th trial failed VI_ERROR_RSRC_BUSY (-1073807246): The resource is valid, but VISA cannot currently access it.
bingo!
2th trial failed VI_ERROR_RSRC_BUSY (-1073807246): The resource is valid, but VISA cannot currently access it.
bingo!
3th trial failed VI_ERROR_RSRC_BUSY (-1073807246): The resource is valid, but VISA cannot currently access it.
bingo!
4th trial failed VI_ERROR_RSRC_BUSY (-1073807246): The resource is valid, but VISA cannot currently access it.
bingo!
bingo!
bingo!
bingo!
bingo!
1th trial failed VI_ERROR

In [11]:
stop_process()